In [4]:
import akshare as ak
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import talib

In [5]:
hk_tickers = ['09988.HK','00700.HK', '01024.HK', '03032.HK']
us_tickers = ["AAPL", "MSFT", "GOOG"]
#"600000.sh"
a_tickers = ["sh600580"]

## 测试抓取东方财富web页面获取股票历史交易价格数据

In [75]:
def stock_zh_a_hist(
    symbol: str = "000001",
    period: str = "daily",
    start_date: str = "19700101",
    end_date: str = "20500101",
    adjust: str = "",
    timeout: float = None,
) -> pd.DataFrame:
    """
    东方财富网-行情首页-沪深京 A 股-每日行情
    https://quote.eastmoney.com/concept/sh603777.html?from=classic
    :param symbol: 股票代码
    :type symbol: str
    :param period: choice of {'daily', 'weekly', 'monthly'}
    :type period: str
    :param start_date: 开始日期
    :type start_date: str
    :param end_date: 结束日期
    :type end_date: str
    :param adjust: choice of {"qfq": "前复权", "hfq": "后复权", "": "不复权"}
    :type adjust: str
    :param timeout: choice of None or a positive float number
    :type timeout: float
    :return: 每日行情
    :rtype: pandas.DataFrame
    """
    market_code = 1 if symbol.startswith("6") else 0
    adjust_dict = {"qfq": "1", "hfq": "2", "": "0"}
    period_dict = {"daily": "101", "weekly": "102", "monthly": "103"}
    url = "https://push2his.eastmoney.com/api/qt/stock/kline/get"
    params = {
        "fields1": "f1,f2,f3,f4,f5,f6",
        "fields2": "f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f116",
        "ut": "7eea3edcaed734bea9cbfc24409ed989",
        "klt": period_dict[period],
        "fqt": adjust_dict[adjust],
        "secid": f"{market_code}.{symbol}",
        "beg": start_date,
        "end": end_date,
    }
    r = requests.get(url, params=params, timeout=timeout)
    data_json = r.json()
    if not (data_json["data"] and data_json["data"]["klines"]):
        return pd.DataFrame()
    temp_df = pd.DataFrame([item.split(",") for item in data_json["data"]["klines"]])
    temp_df["股票代码"] = symbol
    return temp_df

In [76]:
daily_data = stock_zh_a_hist(symbol='600580',start_date='20250415',end_date='20250430')
print(f"600580 daily data: {daily_data.columns}\n {daily_data.head()}")

600580 daily data: Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, '股票代码'], dtype='object')
             0      1      2      3      4       5              6     7      8  \
0  2025-04-15  23.39  23.39  23.85  23.01  722048  1688993865.00  3.57  -0.51   
1  2025-04-16  23.16  22.78  23.45  22.60  625241  1434818061.00  3.63  -2.61   
2  2025-04-17  22.70  22.81  23.30  22.70  555837  1279108369.00  2.63   0.13   
3  2025-04-18  22.81  22.71  22.97  22.54  389961   885921137.00  1.89  -0.44   
4  2025-04-21  22.08  23.33  23.37  22.03  758922  1735628087.00  5.90   2.73   

       9    10    股票代码  
0  -0.12  5.54  600580  
1  -0.61  4.80  600580  
2   0.03  4.27  600580  
3  -0.10  2.99  600580  
4   0.62  5.83  600580  


### 调akshare的stock_zh_a_hist接口获取股票历史价格数据

In [81]:
def get_stock_daily_data(symbol, start_date=None, end_date=None, adjust="qfq"):
    """
    获取股票日K线数据
    :param symbol: 股票代码，如：sh600000 或 sz000001
    :param start_date: 开始日期，如：20210101
    :param end_date: 结束日期，如：20211231
    :param adjust: 复权类型，qfq: 前复权, hfq: 后复权, None: 不复权
    :return: DataFrame
    """
    # 将代码转换为AKShare格式
    # if symbol.startswith('sh') or symbol.startswith('sz'):
    #     stock_code = symbol
    # else:
    #     # 自动判断上海或深圳
    #     if symbol.startswith('6'):
    #         stock_code = f"sh{symbol}"
    #     else:
    #         stock_code = f"sz{symbol}"
    
    # 获取数据
    stock_data = ak.stock_zh_a_hist(symbol=symbol, period="daily", 
                                    start_date=start_date, end_date=end_date, 
                                    adjust=adjust)
    stock_data.columns = ['ticker','date', 'open', 'close', 'high', 'low', 'volume', 'amount', '']
    return stock_data


In [82]:
stock_data = get_stock_daily_data(symbol='600580', start_date='20250415', end_date='20250506')
print(f'stock_data type: {type(stock_data)}')
print(f'stock data len: {len(stock_data)}')
print(f'stock daily data all columns: {stock_data.columns}')
print(f'data: \n {stock_data}')

stock_data type: <class 'pandas.core.frame.DataFrame'>
stock data len: 13
stock daily data all columns: Index(['日期', '股票代码', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额',
       '换手率'],
      dtype='object')
data: 
             日期    股票代码     开盘     收盘     最高     最低      成交量           成交额  \
0   2025-04-15  600580  23.39  23.39  23.85  23.01   722048  1.688994e+09   
1   2025-04-16  600580  23.16  22.78  23.45  22.60   625241  1.434818e+09   
2   2025-04-17  600580  22.70  22.81  23.30  22.70   555837  1.279108e+09   
3   2025-04-18  600580  22.81  22.71  22.97  22.54   389961  8.859211e+08   
4   2025-04-21  600580  22.08  23.33  23.37  22.03   758922  1.735628e+09   
5   2025-04-22  600580  23.28  22.70  23.38  22.69   536935  1.231726e+09   
6   2025-04-23  600580  23.08  24.10  24.16  23.01  1401464  3.318686e+09   
7   2025-04-24  600580  24.11  23.72  24.52  23.57   953194  2.285994e+09   
8   2025-04-25  600580  23.86  23.73  24.09  23.43   650675  1.545819e+09   
9 

#akshare接口get_stock_basic_info获取公司基本信息

In [ ]:
def get_all_stock_code():
    """
    获取A股上市公司基本信息
    :return: DataFrame
    """
    # 获取数据
    stock_info = ak.stock_info_a_code_name()
    return stock_info


In [ ]:
stock_info = get_all_stock_code()
print(f"all stock info size: {len(stock_info)},columns: {stock_info.columns} \n {stock_info.head()}")

all stock info size: 5411,columns: Index(['code', 'name'], dtype='object') 
      code   name
0  000001   平安银行
1  000002  万  科Ａ
2  000004  *ST国华
3  000006   深振业Ａ
4  000007    全新好


#akshare指定股票的公司基本信息

In [28]:
def get_company_info(symbol):
    """
    获取指定股票的公司基本信息
    :param symbol: 股票代码，如：000001
    :return: DataFrame
    """
    # 获取数据
    company_info = ak.stock_individual_info_em(symbol=symbol)
    print(f"company_info columns:{company_info.columns} \n info: {company_info}")

    # 0    最新               25.72
    # 1  股票代码              600580
    # 2  股票简称                卧龙电驱
    # 3   总股本        1302622626.0
    # 4   流通股        1302622626.0
    # 5   总市值  33503453940.719997
    # 6  流通市值  33503453940.719997
    # 7    行业                  电机
    # 8  上市时间            20020606

    """
    获取公司概况和主营业务
    :param symbol: 股票代码，如：000001
    :return: DataFrame
    """
    # 获取数据
    profile = ak.stock_profile_cninfo(symbol=symbol)
    print(f"company profile: {profile.columns}, \n {profile.describe}")

    for col in profile.columns:
        print(f"company profile:\n {col} : {profile[col]}")
    # 主要关注:所属行业, 主营业务
    # company profile: Index(['公司名称', '英文名称', '曾用简称', 'A股代码', 'A股简称', 'B股代码', 'B股简称', 'H股代码', 'H股简称',
    #    '入选指数', '所属市场', '所属行业', '法人代表', '注册资金', '成立日期', '上市日期', '官方网站', '电子邮箱',
    #    '联系电话', '传真', '注册地址', '办公地址', '邮政编码', '主营业务', '经营范围', '机构简介']
    """
    获取公司十大股东数据
    :param symbol: 股票代码，如：000001
    :return: DataFrame
    """
    # TODO: 接口 symbol=sh600000
    if symbol.startswith('sh') or symbol.startswith('sz'):
        stock_code = symbol
    else:
        # 自动判断上海或深圳
        if symbol.startswith('6'):
            stock_code = f"sh{symbol}"
        else:
            stock_code = f"sz{symbol}"
    shareholders = ak.stock_gdfx_top_10_em(symbol=stock_code)
    print(f"company shareholders: {shareholders}")

    # 获取公司股东人数变化
    stock_zh_a_gdhs_detail_em_df = ak.stock_zh_a_gdhs_detail_em(symbol=symbol)
    print(f"公司股东人数变化: {stock_zh_a_gdhs_detail_em_df}")
    # 信息比较详细
    # 公司股东人数变化:    
    # 股东户数统计截止日      区间涨跌幅  股东户数-本次  股东户数-上次  
    # 股东户数-增减  股东户数-增减比例  ...         
    # 总股本       股本变动     股本变动原因    股东户数公告日期      代码    名称


In [29]:
get_company_info(symbol='600580')

company_info columns:Index(['item', 'value'], dtype='object') 
 info:    item               value
0    最新                25.6
1  股票代码              600580
2  股票简称                卧龙电驱
3   总股本        1302622626.0
4   流通股        1302622626.0
5   总市值  33347139225.600002
6  流通市值  33347139225.600002
7    行业                  电机
8  上市时间            20020606
company profile: Index(['公司名称', '英文名称', '曾用简称', 'A股代码', 'A股简称', 'B股代码', 'B股简称', 'H股代码', 'H股简称',
       '入选指数', '所属市场', '所属行业', '法人代表', '注册资金', '成立日期', '上市日期', '官方网站', '电子邮箱',
       '联系电话', '传真', '注册地址', '办公地址', '邮政编码', '主营业务', '经营范围', '机构简介'],
      dtype='object'), 
 <bound method NDFrame.describe of              公司名称                             英文名称  曾用简称    A股代码  A股简称  B股代码  \
0  卧龙电气驱动集团股份有限公司  Wolong Electric Group Co., Ltd.  卧龙电气  600580  卧龙电驱  None   

   B股简称  H股代码  H股简称                           入选指数  ...               官方网站  \
0  None  None  None  ESG 300,国证Ａ指,巨潮小盘,上证380,中证500  ...  www.wolong.com.cn   

                      电子邮箱  

  0%|          | 0/1 [00:00<?, ?it/s]

公司股东人数变化:      股东户数统计截止日      区间涨跌幅  股东户数-本次  股东户数-上次  股东户数-增减  股东户数-增减比例  \
0   2025-03-31  54.101222   250669   143375   107294  74.834525   
1   2024-12-31  35.141509   143375    96779    46596  48.146809   
2   2024-09-30   5.583867    96779   108477   -11698 -10.783853   
3   2024-06-30 -28.106509   108477   111209    -2732  -2.456636   
4   2024-03-31  44.075021   111209    74874    36335  48.528194   
5   2023-12-31  -2.493766    74874    71068     3806   5.355434   
6   2023-09-30 -16.737408    71068    76132    -5064  -6.651605   
7   2023-06-30  15.676960    76132    72910     3222   4.419147   
8   2023-03-31   1.364366    72910    73334     -424  -0.578177   
9   2022-12-31   7.413793    73334    76937    -3603  -4.683052   
10  2022-09-30 -18.715296    76937    69900     7037  10.067239   
11  2022-06-30   9.908537    69900    61617     8283  13.442719   
12  2022-03-31 -28.266812    61617    60900      717   1.177340   
13  2021-12-31  37.622272    60900    67542    -6642

#akshare api接口stock_financial_report_sina获取公司财报信息(不包含很多有价值的财务指标)

In [12]:
def get_stock_financial_income_stmt(stock_symbol):
    """
    获取公司利润表
    :param symbol: 股票代码，如：000001
    :return: DataFrame
    """
    
    try:
        #income = ak.stock_financial_report_sina(stock=stock_symbol, symbol="利润表")
        # income_ths_abstract = ak.stock_financial_abstract_ths(symbol=stock_symbol)
        # income_abstract = ak.stock_financial_abstract(symbol=stock_symbol)
        income = ak.stock_financial_benefit_ths(symbol=stock_symbol)
        print(f"company income {income.columns.size} {income.columns}:\n {income.head()}")
    except Exception as e:
        print(f"company income: {e}")

def get_stock_financial_balance_sheet(stock_symbol):
    """
    获取公司资产负债表
    :param symbol: 股票代码，如：000001
    :return: DataFrame
    """
    try:
        #balance = ak.stock_financial_report_sina(stock=stock_symbol, symbol="资产负债表")
        balance = ak.stock_financial_debt_ths(symbol=stock_symbol)
        print(f"company balance {balance.columns.size} {balance.columns}:\n {balance.head()}")
    except Exception as e:
        print(f"company balance: {e}")

def get_stock_financial_cashflow_sheet(stock_symbol):

    """
    获取公司现金流量表
    :param symbol: 股票代码，如：000001
    :return: DataFrame
    """
    try:
        #cash_flow = ak.stock_financial_report_sina(stock=stock_symbol, symbol="现金流量表")
        cash_flow = ak.stock_financial_cash_ths(symbol=stock_symbol)
        print(f"company cash_flow {cash_flow.columns.size} {cash_flow.columns}: \n {cash_flow.head()}")
    except Exception as e:
        print(f"company cash_flow: {e}")


In [13]:
get_stock_financial_income_stmt('600580')

company income 46 Index(['报告期', '报表核心指标', '*净利润', '*营业总收入', '*营业总成本', '*归属于母公司所有者的净利润',
       '*扣除非经常性损益后的净利润', '报表全部指标', '一、营业总收入', '其中：营业收入', '二、营业总成本', '其中：营业成本',
       '营业税金及附加', '销售费用', '管理费用', '研发费用', '财务费用', '其中：利息费用', '利息收入', '资产减值损失',
       '信用减值损失', '加：公允价值变动收益', '投资收益', '其中：联营企业和合营企业的投资收益', '资产处置收益', '其他收益',
       '三、营业利润', '加：营业外收入', '其中：非流动资产处置利得', '减：营业外支出', '其中：非流动资产处置损失',
       '四、利润总额', '减：所得税费用', '五、净利润', '（一）持续经营净利润', '归属于母公司所有者的净利润', '少数股东损益',
       '扣除非经常性损益后的净利润', '六、每股收益', '（一）基本每股收益', '（二）稀释每股收益', '七、其他综合收益',
       '归属母公司所有者的其他综合收益', '八、综合收益总额', '归属于母公司股东的综合收益总额', '归属于少数股东的综合收益总额'],
      dtype='object'):
           报告期 报表核心指标   *净利润   *营业总收入   *营业总成本 *归属于母公司所有者的净利润 *扣除非经常性损益后的净利润  \
0  2025-03-31         2.77亿   40.39亿   37.82亿          2.68亿          2.35亿   
1  2024-12-31         8.32亿  162.47亿  155.40亿          7.93亿          6.35亿   
2  2024-09-30         6.78亿  121.94亿  115.56亿          6.38亿          5.72亿   
3  2024-06-30         4.09亿   79.78亿   

In [14]:
get_stock_financial_balance_sheet('600580')

company balance 77 Index(['报告期', '报表核心指标', '*所有者权益（或股东权益）合计', '*资产合计', '*负债合计', '*归属于母公司所有者权益合计',
       '报表全部指标', '流动资产', '货币资金', '交易性金融资产', '应收票据及应收账款', '其中：应收票据', '应收账款',
       '预付款项', '其他应收款合计', '其他应收款', '存货', '其他流动资产', '总现金', '流动资产合计', '非流动资产',
       '可供出售金融资产', '长期股权投资', '其他非流动金融资产', '投资性房地产', '固定资产合计', '其中：固定资产',
       '固定资产清理', '在建工程合计', '其中：在建工程', '工程物资', '无形资产', '商誉', '长期待摊费用',
       '递延所得税资产', '其他非流动资产', '非流动资产合计', '资产合计', '流动负债', '短期借款',
       '以公允价值计量且其变动计入当期损益的金融负债', '衍生金融负债', '应付票据及应付账款', '其中：应付票据', '应付账款',
       '预收款项', '合同负债', '应付职工薪酬', '应交税费', '其他应付款合计', '其中：应付利息', '应付股利', '其他应付款',
       '一年内到期的非流动负债', '其他流动负债', '流动负债合计', '非流动负债', '长期借款', '长期应付款合计',
       '其中：长期应付款', '预计负债', '递延所得税负债', '递延收益-非流动负债', '其他非流动负债', '非流动负债合计',
       '负债合计', '所有者权益（或股东权益）', '实收资本（或股本）', '资本公积', '减：库存股', '其他综合收益', '盈余公积',
       '未分配利润', '归属于母公司所有者权益合计', '少数股东权益', '所有者权益（或股东权益）合计',
       '负债和所有者权益（或股东权益）合计'],
      dtype='object'):
           报告期 报表核心指标 *所有者权益（或股东权益）合计    *资产合计    *

In [15]:
get_stock_financial_cashflow_sheet('600580')

company cash_flow 73 Index(['报告期', '报表核心指标', '*现金及现金等价物净增加额', '*经营活动产生的现金流量净额', '*投资活动产生的现金流量净额',
       '*筹资活动产生的现金流量净额', '*期末现金及现金等价物余额', '报表全部指标', '一、经营活动产生的现金流量',
       '销售商品、提供劳务收到的现金', '收到的税费与返还', '收到其他与经营活动有关的现金', '经营活动现金流入小计',
       '购买商品、接受劳务支付的现金', '支付给职工以及为职工支付的现金', '支付的各项税费', '支付其他与经营活动有关的现金',
       '经营活动现金流出小计', '经营活动产生的现金流量净额', '二、投资活动产生的现金流量', '收回投资收到的现金',
       '取得投资收益收到的现金', '处置固定资产、无形资产和其他长期资产收回的现金净额', '处置子公司及其他营业单位收到的现金净额',
       '收到其他与投资活动有关的现金', '投资活动现金流入小计', '购建固定资产、无形资产和其他长期资产支付的现金', '投资支付的现金',
       '取得子公司及其他营业单位支付的现金净额', '支付其他与投资活动有关的现金', '投资活动现金流出小计', '投资活动产生的现金流量净额',
       '三、筹资活动产生的现金流量', '吸收投资收到的现金', '其中：子公司吸收少数股东投资收到的现金', '取得借款收到的现金',
       '收到其他与筹资活动有关的现金', '筹资活动现金流入小计', '偿还债务支付的现金', '分配股利、利润或偿付利息支付的现金',
       '其中：子公司支付给少数股东的股利、利润', '支付其他与筹资活动有关的现金', '筹资活动现金流出小计', '筹资活动产生的现金流量净额',
       '四、汇率变动对现金及现金等价物的影响', '五、现金及现金等价物净增加额', '加：期初现金及现金等价物余额',
       '六、期末现金及现金等价物余额', '补充资料：', '1、将净利润调节为经营活动现金流量：', '净利润', '加：资产减值准备',
       '固定资产折旧、油气资产折耗、生产性生物资

### akshare stock_financial_analysis_indicator 获取财务指标(该接口有问题)

In [33]:
def get_company_financial_analysis(symbol):
    """
    获取公司财务分析指标
    :param symbol: 股票代码，如：000001
    :return: DataFrame
    """
    #ak.stock_financial_abstract(symbol=symbol)
    analysis = ak.stock_financial_analysis_indicator(symbol=symbol, start_year='2020')
    print(f"company financial analysis: {analysis}")

In [40]:
get_company_financial_analysis('600580')

company financial analysis: Empty DataFrame
Columns: []
Index: []


In [6]:
from datetime import datetime
from io import StringIO
import requests
from bs4 import BeautifulSoup

### stock_financial_analysis_indicator 获取财务指标(重写可获取)

In [7]:
def stock_financial_analysis_indicator(
    symbol: str = "600004", start_year: str = "2020"
) -> pd.DataFrame:
    """
    新浪财经-财务分析-财务指标
    https://money.finance.sina.com.cn/corp/go.php/vFD_FinancialGuideLine/stockid/600004/ctrl/2019/displaytype/4.phtml
    :param symbol: 股票代码
    :type symbol: str
    :param start_year: 开始年份
    :type start_year: str
    :return: 新浪财经-财务分析-财务指标
    :rtype: pandas.DataFrame
    """
    url = (
        f"https://money.finance.sina.com.cn/corp/go.php/vFD_FinancialGuideLine/"
        f"stockid/{symbol}/ctrl/2020/displaytype/4.phtml"
    )
    r = requests.get(url)
    soup = BeautifulSoup(r.text, features="lxml")
    year_context = soup.find(attrs={"id": "con02-1"}).find("table").find_all("a")
    year_list = [item.text for item in year_context]
    print(f"year_list from url: {year_list}")
    if start_year in year_list:
        year_list = year_list[: year_list.index(start_year) + 1]
        print(f"year_list: {year_list}")
    else:
        return pd.DataFrame()
    out_df = pd.DataFrame()
    print(f"continue request year finance....")

    for year_item in year_list:
        url = (
            f"https://money.finance.sina.com.cn/corp/go.php/vFD_FinancialGuideLine/"
            f"stockid/{symbol}/ctrl/{year_item}/displaytype/4.phtml"
        )
        print(f"request {symbol} {year_item} url: {url}")
        r = requests.get(url)
        #print(f"request {symbol} {year_item} text: {StringIO(r.text)}")
        # 保存每年的HTML响应
        year_html_file = f"financial_html_{symbol}_{year_item}.html"
        with open(year_html_file, "w", encoding="utf-8") as f:
            f.write(r.text)
        
        html_tables = pd.read_html(StringIO(r.text))
         # 保存所有表格到CSV文件
        for i, table in enumerate(html_tables):
            if i == 12:
                table_file = f"financial_table_{symbol}_{year_item}_table{i}.csv"
                table.to_csv(table_file, index=False, encoding="utf-8")
            
        temp_df = pd.read_html(StringIO(r.text))[12].iloc[:, :-1]
        temp_df.columns = temp_df.iloc[0, :]
        print(f'df columns: {temp_df.columns}')
        temp_df = temp_df.iloc[1:, :]
        big_df = pd.DataFrame()
        indicator_list = [
            "每股指标",
            "盈利能力",
            "成长能力",
            "营运能力",
            "偿债及资本结构",
            "现金流量",
            "其他指标",
        ]
        for i in range(len(indicator_list)):
            if i == 6:
                inner_df = temp_df[
                    temp_df.loc[
                        temp_df.iloc[:, 0].str.find(indicator_list[i]) == 0, :
                    ].index[0] :
                ].T
            else:
                inner_df = temp_df[
                    temp_df.loc[
                        temp_df.iloc[:, 0].str.find(indicator_list[i]) == 0, :
                    ].index[0] : temp_df.loc[
                        temp_df.iloc[:, 0].str.find(indicator_list[i + 1]) == 0, :
                    ].index[0]
                    - 1
                ].T
            inner_df = inner_df.reset_index(drop=True)
            big_df = pd.concat(objs=[big_df, inner_df], axis=1)
        big_df.columns = big_df.iloc[0, :].tolist()
        big_df = big_df.iloc[1:, :]
        big_df.index = temp_df.columns.tolist()[1:]
        out_df = pd.concat(objs=[out_df, big_df])

    out_df.dropna(inplace=True)
    out_df.reset_index(inplace=True)
    out_df.rename(columns={"index": "日期"}, inplace=True)
    out_df.sort_values(by=["日期"], ignore_index=True, inplace=True)
    out_df["日期"] = pd.to_datetime(out_df["日期"], errors="coerce").dt.date
    for item in out_df.columns[1:]:
        out_df[item] = pd.to_numeric(out_df[item], errors="coerce")
    return out_df


In [8]:
df = stock_financial_analysis_indicator('600580')
print(f"company financial analysis:{df.columns} \n {df.head()}")

year_list from url: ['2025', '2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000', '1999']
year_list: ['2025', '2024', '2023', '2022', '2021', '2020']
continue request year finance....
request 600580 2025 url: https://money.finance.sina.com.cn/corp/go.php/vFD_FinancialGuideLine/stockid/600580/ctrl/2025/displaytype/4.phtml
df columns: Index(['报告日期', '2025-03-31', nan, nan, nan], dtype='object', name=0)
request 600580 2024 url: https://money.finance.sina.com.cn/corp/go.php/vFD_FinancialGuideLine/stockid/600580/ctrl/2024/displaytype/4.phtml
df columns: Index(['报告日期', '2024-12-31', '2024-09-30', '2024-06-30', '2024-03-31'], dtype='object', name=0)
request 600580 2023 url: https://money.finance.sina.com.cn/corp/go.php/vFD_FinancialGuideLine/stockid/600580/ctrl/2023/displaytype/4.phtml
df columns: Index(['报告日期', '2023-12-31', '2023-09-30', '2023-06

In [ ]:
def get_company_import_info(symbol):
    metrics = {}
    # 获取公司基本信息
    try:
        company_info = ak.stock_individual_info_em(symbol=symbol)
        if not company_info.empty:
            # 转换为字典方便查询
            info_dict = {}
            for i in range(len(company_info)):
                info_dict[company_info.iloc[i, 0]] = company_info.iloc[i, 1]
                print(f"""company info: {company_info.iloc[i, 0]}: {company_info.iloc[i, 1]}""")
            
            # # 市值 (总市值)
            # if "总市值" in info_dict:
            #     metrics["market_cap"] = float(info_dict["总市值"])
            
            # # 市盈率
            # if "市盈率" in info_dict:
            #     metrics["price_to_earnings_ratio"] = float(info_dict["市盈率"])
    except Exception as e:
        print(f"获取公司基本信息失败: {e}")
    return metrics

In [ ]:
metrics = get_company_import_info('600580')
print(f"company metrics: {metrics}")

company info: 最新: 26.04
company info: 股票代码: 600580
company info: 股票简称: 卧龙电驱
company info: 总股本: 1302622626.0
company info: 流通股: 1302622626.0
company info: 总市值: 33920293181.039997
company info: 流通市值: 33920293181.039997
company info: 行业: 电机
company info: 上市时间: 20020606
company metrics: {'market_cap': 33920293181.039997}


### 获取行业板块的信息

In [23]:
def get_industry_classification():
    """
    获取行业分类数据
    :return: DataFrame
    """
    # 新浪行业分类
    industry = ak.stock_sector_spot()
    # columns: 
    # label    板块  公司家数...
    # new_dzxx  电子信息   24 ....
    return industry

In [26]:
industry = get_industry_classification()
print(f"all industry:{len(industry)}, \n columns:{industry.columns} \n {industry.head()}")

all industry:49, 
 columns:Index(['label', '板块', '公司家数', '平均价格', '涨跌额', '涨跌幅', '总成交量', '总成交额', '股票代码',
       '个股-涨跌幅', '个股-当前价', '个股-涨跌额', '股票名称'],
      dtype='object') 
       label    板块  公司家数       平均价格       涨跌额       涨跌幅        总成交量  \
0  new_blhy  玻璃行业    19  10.723684 -0.153158 -1.408110   296606369   
1  new_cbzz  船舶制造     8  12.561250 -0.105000 -0.828975   695005999   
2  new_cmyl  传媒娱乐    40   8.375833 -0.126389 -1.486539   918421050   
3  new_dlhy  电力行业    62   7.590667  0.010667  0.140721  3711182141   
4  new_dqhy  电器行业    58  13.439259 -0.222037 -1.625300  1278707781   

          总成交额      股票代码  个股-涨跌幅  个股-当前价  个股-涨跌额   股票名称  
0   2795181336  sh603021   3.939    3.43    0.13   ST华鹏  
1   5911821493  sz300123   1.698    6.59    0.11   亚光科技  
2   6748667100  sz000504   5.055    6.65    0.32  *ST生物  
3  22814202906  sz000767   9.929    3.10    0.28   晋控电力  
4  15138889678  sz002164   6.076   10.30    0.59   宁波东力  


### 获取某个行业板块的所有股票

In [27]:
industry_detail = ak.stock_sector_detail(sector='new_dlhy')
print(f"industry_detail:{len(industry_detail)}, \n columns:{industry_detail.columns} \n {industry_detail.head()}")

100%|██████████| 1/1 [00:00<00:00,  5.38it/s]

industry_detail:62, 
 columns:Index(['symbol', 'code', 'name', 'trade', 'pricechange', 'changepercent',
       'buy', 'sell', 'settlement', 'open', 'high', 'low', 'volume', 'amount',
       'ticktime', 'per', 'pb', 'mktcap', 'nmc', 'turnoverratio'],
      dtype='object') 
      symbol    code  name  trade  pricechange  changepercent   buy  sell  \
0  sh600011  600011  华能国际   7.45         0.13          1.776  7.44  7.45   
1  sh600021  600021  上海电力   9.15         0.09          0.993  9.14  9.15   
2  sh600023  600023  浙能电力   5.31        -0.02         -0.375  5.31  5.32   
3  sh600027  600027  华电国际   5.87         0.11          1.910  5.87  5.88   
4  sh600098  600098  广州发展   6.51         0.02          0.308  6.51  6.52   

   settlement  open  high   low     volume     amount  ticktime     per  \
0        7.32  7.32  7.55  7.32   86181307  642315858  15:00:01  16.196   
1        9.06  9.05  9.29  9.02   52844134  482737182  15:00:03  14.825   
2        5.33  5.33  5.37  5.31   50165511  

#akshare stock_board_concept_name_em 获取A股所有概念板块

In [15]:
def get_concept_sectors():
    """
    获取概念板块数据
    :return: DataFrame
    """
    # 获取数据
    concept = ak.stock_board_concept_name_em()
    
    # columns:
    # 排名      板块名称    板块代码      最新价    涨跌额   涨跌幅            
    # 总市值   换手率  上涨家数  下跌家数  领涨股票  领涨股票-涨跌幅
    # eg: 人形机器人  BK1184
    return concept


In [16]:
concept = get_concept_sectors()
print(f"concept columns: {concept.columns}")
print(f"all concept: {concept.head()}")

  0%|          | 0/4 [00:00<?, ?it/s]

concept columns: Index(['排名', '板块名称', '板块代码', '最新价', '涨跌额', '涨跌幅', '总市值', '换手率', '上涨家数', '下跌家数',
       '领涨股票', '领涨股票-涨跌幅'],
      dtype='object')
all concept:    排名      板块名称    板块代码       最新价     涨跌额   涨跌幅           总市值    换手率  上涨家数  \
0   1      昨日连板  BK0816     60.81    2.41  4.13   61197044000  14.95    11   
1   2  昨日连板_含一字  BK1051  11519.45  386.69  3.47  131084708000  14.81    27   
2   3  昨日涨停_含一字  BK1050    426.00    5.09  1.21  785001904000  10.92    59   
3   4       ST股  BK0511   5491.82   60.14  1.11  662451376000   3.00   123   
4   5      代糖概念  BK0938   1003.64    9.95  1.00  143008722000   2.33     7   

   下跌家数  领涨股票  领涨股票-涨跌幅  
0     4  成飞集成     10.02  
1     7  成飞集成     10.02  
2    44  成飞集成     10.02  
3    59  ST香雪      8.70  
4     8  百龙创园      8.33  


#akshare获取某个概念板块的所有股票

In [20]:
def get_concept_sector_stocks(concept_code):
    """
    获取指定概念板块的股票数据
    :param concept_code: 概念板块代码，如：BK1184
    :return: DataFrame
    """
    # BK1184
    # 获取数据
    concept_stocks = ak.stock_board_concept_cons_em(symbol=concept_code)
    # 获取概念板块的历史数据
    #concept_hist = ak.stock_board_concept_hist_em(symbol=concept_code, start_date='20250501', end_date='20250510')
#     print(f"""concept sector stocks: {concept_code}, total: {len(concept_stocks)} \n
#           data: {concept_stocks.head(10)}""")
    return concept_stocks

In [21]:
concept_stocks = get_concept_sector_stocks('BK1184')
print(f"""concept sector stocks: BK1184, total: {len(concept_stocks)}, \n
          columns: {concept_stocks.columns} \n
          data: {concept_stocks.head(10)}""")

  0%|          | 0/1 [00:00<?, ?it/s]

concept sector stocks: BK1184, total: 115, 

          columns: Index(['序号', '代码', '名称', '最新价', '涨跌幅', '涨跌额', '成交量', '成交额', '振幅', '最高', '最低',
       '今开', '昨收', '换手率', '市盈率-动态', '市净率'],
      dtype='object') 

          data:    序号      代码     名称    最新价   涨跌幅   涨跌额      成交量           成交额     振幅     最高  \
0   1  001319   铭科精技  28.14  3.84  1.04   129046  3.567335e+08   8.71  28.61   
1   2  002031   巨轮智能   9.41  2.73  0.25  3706498  3.545459e+09   9.06   9.89   
2   3  002965   祥鑫科技  56.78  2.29  1.27   249718  1.393539e+09   9.58  58.92   
3   4  000887   中鼎股份  19.43  2.26  0.43  1200475  2.347416e+09   6.95  20.32   
4   5  605555   德昌股份  21.64  1.31  0.28   113248  2.429514e+08   2.95  21.76   
5   6  301236   软通动力  59.65  1.20  0.71   554580  3.363295e+09   6.18  62.65   
6   7  000333   美的集团  75.80  1.07  0.80   245639  1.861536e+09   1.76  76.03   
7   8  688165  埃夫特-U  25.56  0.99  0.25   269413  6.779704e+08  10.23  26.50   
8   9  000157   中联重科   7.63  0.39  0.03   361203  2.74

# 计算常用技术指标

In [ ]:
def calculate_technical_indicators(price_data):
    """
    计算常用技术指标
    :param stock_data: 股票数据，包含OHLC
    :return: DataFrame，添加了技术指标
    """
    # 确保列名一致
    if '收盘' in price_data.columns:
        close = price_data['收盘'].values
        high = price_data['最高'].values
        low = price_data['最低'].values
        open_price = price_data['开盘'].values
        volume = price_data['成交量'].values
    else:
        close = price_data['close'].values
        high = price_data['high'].values
        low = price_data['low'].values
        open_price = price_data['open'].values
        volume = price_data['volume'].values if 'volume' in price_data.columns else None
    
    # 计算移动平均线
    price_data['MA5'] = talib.SMA(close, timeperiod=5)
    price_data['MA10'] = talib.SMA(close, timeperiod=10)
    price_data['MA20'] = talib.SMA(close, timeperiod=20)
    price_data['MA30'] = talib.SMA(close, timeperiod=30)
    price_data['MA60'] = talib.SMA(close, timeperiod=60)
    
    # 计算MACD
    macd, macd_signal, macd_hist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    price_data['MACD'] = macd
    price_data['MACD_SIGNAL'] = macd_signal
    price_data['MACD_HIST'] = macd_hist
    
    # 计算RSI
    price_data['RSI6'] = talib.RSI(close, timeperiod=6)
    price_data['RSI12'] = talib.RSI(close, timeperiod=12)
    price_data['RSI24'] = talib.RSI(close, timeperiod=24)
    
    # 计算布林带
    upper, middle, lower = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    price_data['BOLL_UPPER'] = upper
    price_data['BOLL_MIDDLE'] = middle
    price_data['BOLL_LOWER'] = lower
    
    # 计算KDJ
    price_data['K'], price_data['D'] = talib.STOCH(high, low, close, 
                                                  fastk_period=9, 
                                                  slowk_period=3, 
                                                  slowk_matype=0, 
                                                  slowd_period=3, 
                                                  slowd_matype=0)
    price_data['J'] = 3 * price_data['K'] - 2 * price_data['D']
    
    # 如果有成交量数据，计算成交量指标
    if volume is not None and not np.isnan(volume).all():
        # 计算OBV
        price_data['OBV'] = talib.OBV(close, volume)
        
        # 计算成交量移动平均
        price_data['VOLUME_MA5'] = talib.SMA(volume, timeperiod=5)
        price_data['VOLUME_MA10'] = talib.SMA(volume, timeperiod=10)
    
    return price_data



# akshare获取港股日线历史交易价格数据

In [ ]:
def get_hk_stock_daily_history(symbol, start_date=None, end_date=None, adjust="qfq"):
    """
    获取港股交易行情数据
    :param symbol: 股票代码，如：000001
    :param start_date: 开始日期，如：20210101
    :param end_date: 结束日期，如：20211231
    """
    hk_data = ak.stock_hk_hist(symbol=symbol, period="daily", start_date=start_date, end_date=end_date, adjust=adjust)
    print(f"hk_data: {hk_data}")
    # columns: 
    # 日期     开盘     收盘     最高     最低        成交量           
    # 成交额    振幅   涨跌幅  涨跌额   换手率
    return hk_data